In [8]:
import numpy as np
from functools import lru_cache
from typing import Dict, List, Tuple
import time

class SingleObjectDPSolver:
    """
    An exact Dynamic Programming solver for a single-object search problem.

    This class uses backward recursion to find the optimal policy based on a
    marginal prior distribution for a single target.
    """

    def __init__(self, n: int, T: int, p0_marginal: np.ndarray, gamma1: np.ndarray, c: np.ndarray):
        """
        Initializes the solver with the single-object problem parameters.

        Args:
            n (int): The number of cells.
            T (int): The time horizon.
            p0_marginal (np.ndarray): The n-element marginal prior probability vector for O1.
            gamma1 (np.ndarray): Miss-detection rates for O1.
        """
        self.n = n
        self.T = T
        self.p0_marginal = p0_marginal
        self.gamma1 = gamma1
        self.c = c

        # Data structures to store the results
        self.J_values: Dict[int, Dict[Tuple[int, ...], float]] = {}  # Value function J_t(z_t)
        self.Policy: Dict[int, Dict[Tuple[int, ...], int]] = {}      # Policy mu_t(z_t)

    @staticmethod
    @lru_cache(maxsize=None) # Memoization for performance
    def _generate_z_vectors(t: int, n: int) -> List[Tuple[int, ...]]:
        """Recursively generates all state vectors z_t where sum(z_i) = t."""
        if n == 1:
            return [(t,)]

        vectors = []
        for i in range(t + 1):
            for sub_vector in SingleObjectDPSolver._generate_z_vectors(t - i, n - 1):
                vectors.append((i,) + sub_vector)
        return vectors

    def _calculate_posterior(self, z_vector: np.ndarray) -> np.ndarray:
        """Calculates the posterior belief p(O1=i | z)."""
        g1_z = np.power(self.gamma1, z_vector)
        numerator = self.p0_marginal * g1_z
        norm = np.sum(numerator)
        return numerator / norm if norm > 0 else numerator

    def solve(self):
        """
        Executes the backward recursion to solve the DP problem.
        """
        # print("Starting Single-Object DP solver...")
        # --- Initialization at T ---
        # print(f"Initializing for T={self.T}...")
        self.J_values[self.T] = {}
        z_vectors_T = self._generate_z_vectors(self.T, self.n)
        for z_T in z_vectors_T:
            self.J_values[self.T][z_T] = 0.0

        # --- Backward Recursion ---
        for t in range(self.T - 1, -1, -1):
            start_time = time.time()
            J_t = {}
            policy_t = {}
            z_vectors_t = self._generate_z_vectors(t, self.n)

            for z_t_tuple in z_vectors_t:
                z_t = np.array(z_t_tuple)
                action_values = []

                # Calculate current belief vector
                belief_t = self._calculate_posterior(z_t)

                # Iterate over all possible actions
                for a_t in range(self.n):
                    # Probability of finding the target if we search cell a_t
                    p_success = (1 - self.gamma1[a_t]) * belief_t[a_t]
                    p_fail = 1 - ((1 - self.gamma1[a_t]) * belief_t[a_t])

                    # Get the value of the state we transition to upon failure
                    next_z_tuple = tuple(z_t + np.eye(self.n, dtype=int)[a_t])
                    val_if_fail = self.J_values[t + 1][next_z_tuple]

                    # Expected value for this action (Bellman equation)
                    # Reward for success is 1, reward for failure is 0
                    expected_value = (p_success * 1.0) - self.c[a_t] + (p_fail * val_if_fail)
                    action_values.append(expected_value)

                # Find best action and store value/policy
                best_value = np.max(action_values)
                best_action = np.argmax(action_values)
                J_t[z_t_tuple] = best_value
                policy_t[z_t_tuple] = best_action

            self.J_values[t] = J_t
            self.Policy[t] = policy_t
            end_time = time.time()
            # print(f"Completed t={t}. Found {len(z_vectors_t)} states. Took {end_time - start_time:.2f}s.")

        # print("DP solver finished.")

    def get_optimal_value(self) -> float:
        """Returns the optimal value J(z_0)."""
        initial_z = tuple([0] * self.n)
        return self.J_values[0][initial_z]

# Function to run a single episode for the single-object solver
def run_single_object_episode(n: int, T: int, p0_marginal: np.ndarray, gamma1: np.ndarray, c: np.ndarray, policy: Dict, episode_seed: int) -> Tuple[bool, int, float, int]:
    """
    Simulates a single episode using a given single-object DP policy.
    Returns success status, time of detection (or -1 if failed), accumulated reward, and episode length.
    """
    # Create a local RNG for this episode for statistical independence
    rng = np.random.default_rng(episode_seed)

    # Secretly determine the true location of O1
    if np.sum(p0_marginal) == 0:
         true_pos_o1 = -1
    else:
        true_pos_o1 = rng.choice(n, p=p0_marginal / np.sum(p0_marginal))

    z_vector = np.zeros(n, dtype=int)
    accumulated_reward = 0.0
    detection_time = -1
    episode_length = 0

    for t in range(T):
        episode_length += 1  # Increment episode length at each time step
        state = tuple(z_vector)
        if t not in policy or state not in policy[t]:
            # This case should ideally not happen if the policy is complete for the horizon
            print(f"Warning: Policy not found for state {state} at time {t}. Using greedy action.")
            # Fallback to greedy action if policy is missing
            belief = SingleObjectDPSolver(n, T, p0_marginal, gamma1, c)._calculate_posterior(z_vector)
            action = np.argmax((1 - gamma1) * belief - c)
        else:
             action = policy[t][state]

        # Deduct cost for the action
        accumulated_reward -= c[action]

        # Simulate outcome
        if action == true_pos_o1 and rng.random() > gamma1[action]: # Use local RNG
            accumulated_reward += 1.0 # Reward for finding O1
            detection_time = t + 1 # Time is absolute
            return True, detection_time, accumulated_reward, episode_length # Mission Success!

        # Update state
        z_vector[action] += 1

    return False, detection_time, accumulated_reward, episode_length # Mission Failed


if __name__ == '__main__':
    # --- Problem Definition ---
    NUM_CELLS = 5
    TIME_HORIZON = 10
    NUM_EPISODES_PER_SEED = 100  # Number of episodes to run for each seed
    NUM_SEEDS = 100  # Number of different seeds to run

    # The original JOINT prior matrix
    prior_joint = np.array([
        [0.152,  0.0039, 0.003,  0.0108, 0.011],
        [0.0038, 0.0052, 0.117,  0.0162, 0.165],
        [0.0057, 0.195,  0.015,  0.009,  0.011],
        [0.0038, 0.0091, 0.0075, 0.027,  0.011],
        [0.0247, 0.0468, 0.0075, 0.117,  0.022]
    ])

    # Miss-detection rates for O1
    gammas1 = np.array([0.8, 0.65, 0.82, 0.75, 0.7])
    c = np.array([0.15,0.2,0.25,0.1,0.2])

    # --- KEY STEP: Calculate the marginal prior for O1 ---
    # We sum the probabilities across the columns (axis=1) for each row
    p0_marginal_o1 = np.sum(prior_joint, axis=1)

    print("--- Calculating Benchmark for Single-Object Search (Ignoring O2) ---")
    print(f"Original Joint Prior Shape: {prior_joint.shape}")
    print(f"Calculated Marginal Prior for O1 (p(O1)): {np.round(p0_marginal_o1, 4)}")
    print("-" * 20)

    all_success_rates = []
    all_detection_times = []
    all_rewards = []
    all_episode_lengths = [] # New list to store episode lengths
    total_experiment_start_time = time.time()

    print(f"\n--- Running Empirical Experiment for Single-Object DP Policy over Multiple Seeds ---")
    print(f"Number of seeds: {NUM_SEEDS}")
    print(f"Number of evaluation episodes per seed: {NUM_EPISODES_PER_SEED}")
    print("-" * 20)

    solver = SingleObjectDPSolver(n=NUM_CELLS, T=TIME_HORIZON, p0_marginal=p0_marginal_o1, gamma1=gammas1, c=c)
    solver.solve()
    optimal_policy = solver.Policy

    for seed in range(NUM_SEEDS):
        print(f"\n--- Running with Seed: {seed} ---")
        # np.random.seed(seed) # Removed global seeding here to rely on per-episode seeding

        # --- Solve the problem (Policy calculation) ---
        # The policy depends only on the parameters and the seed for sampling true location,
        # but we recalculate it per seed to isolate seed effects on the policy calculation itself if any
        # (though for exact DP with fixed parameters, it should be the same)


        # --- Run the Experiment for this seed ---
        num_successes = 0
        detection_times_this_seed = []
        rewards_this_seed = []
        episode_lengths_this_seed = [] # New list for this seed
        start_time_this_seed = time.time()

        for i in range(NUM_EPISODES_PER_SEED):
            episode_unique_seed = seed * NUM_EPISODES_PER_SEED + i
            success, detection_time, reward, episode_length = run_single_object_episode(
                NUM_CELLS, TIME_HORIZON, p0_marginal_o1, gammas1, c, optimal_policy, episode_unique_seed
            )
            rewards_this_seed.append(reward)
            episode_lengths_this_seed.append(episode_length) # Append episode length

            if success:
                num_successes += 1
                detection_times_this_seed.append(detection_time)

        end_time_this_seed = time.time()
        success_rate_this_seed = num_successes / NUM_EPISODES_PER_SEED
        all_success_rates.append(success_rate_this_seed)
        all_detection_times.extend(detection_times_this_seed)
        all_rewards.extend(rewards_this_seed)
        all_episode_lengths.extend(episode_lengths_this_seed) # Extend overall list

        print(f"--- Seed {seed} Results ---")
        print(f"Success Rate: {success_rate_this_seed:.4f} ({success_rate_this_seed*100:.2f}%)")
        if detection_times_this_seed:
            print(f"Average Detection Time (Successful Episodes): {np.mean(detection_times_this_seed):.2f}")
        else:
            print("Average Detection Time (Successful Episodes): N/A (no successes)")
        print(f"Average Episode Reward: {np.mean(rewards_this_seed):.4f}")
        print(f"Average Episode Length: {np.mean(episode_lengths_this_seed):.2f}") # Display average episode length
        print(f"Time taken for this seed: {end_time_this_seed - start_time_this_seed:.2f} seconds")

    total_experiment_end_time = time.time()

    # --- Display Final Results ---
    mean_success_rate = np.mean(all_success_rates)
    std_success_rate = np.std(all_success_rates)
    mean_detection_time = np.mean(all_detection_times) if all_detection_times else -1
    std_detection_time = np.std(all_detection_times) if all_detection_times else -1
    mean_reward = np.mean(all_rewards)
    std_reward = np.std(all_rewards)
    mean_episode_length = np.mean(all_episode_lengths) # Calculate overall average episode length
    std_episode_length = np.std(all_episode_lengths) # Calculate overall standard deviation of episode length


    print("\n--- Overall Single-Object DP Policy Results ---")
    print(f"Total experiment time across {NUM_SEEDS} seeds: {total_experiment_end_time - total_experiment_start_time:.2f} seconds")
    print(f"Average Success Rate over {NUM_SEEDS} seeds and {NUM_EPISODES_PER_SEED} episodes each: {mean_success_rate:.4f} ({mean_success_rate*100:.2f}%)")
    print(f"Standard Deviation of Success Rate: {std_success_rate:.4f}")
    if all_detection_times:
        print(f"Average Detection Time (Successful Episodes) across all seeds: {mean_detection_time:.2f}")
        print(f"Standard Deviation of Detection Time: {std_detection_time:.2f}")
    else:
        print("Average Detection Time (Successful Episodes) across all seeds: N/A (no successes)")

    print(f"Average Episode Reward across all seeds: {mean_reward:.4f}")
    print(f"Standard Deviation of Reward: {std_reward:.4f}")
    print(f"Average Episode Length across all seeds: {mean_episode_length:.2f}") # Display overall average episode length
    print(f"Standard Deviation of Episode Length: {std_episode_length:.2f}") # Display overall standard deviation of episode length


--- Calculating Benchmark for Single-Object Search (Ignoring O2) ---
Original Joint Prior Shape: (5, 5)
Calculated Marginal Prior for O1 (p(O1)): [0.1807 0.3072 0.2357 0.0584 0.218 ]
--------------------

--- Running Empirical Experiment for Single-Object DP Policy over Multiple Seeds ---
Number of seeds: 100
Number of evaluation episodes per seed: 100
--------------------

--- Running with Seed: 0 ---
--- Seed 0 Results ---
Success Rate: 0.3600 (36.00%)
Average Detection Time (Successful Episodes): 2.64
Average Episode Reward: -0.6775
Average Episode Length: 7.35
Time taken for this seed: 0.01 seconds

--- Running with Seed: 1 ---
--- Seed 1 Results ---
Success Rate: 0.2900 (29.00%)
Average Detection Time (Successful Episodes): 2.97
Average Episode Reward: -0.8325
Average Episode Length: 7.96
Time taken for this seed: 0.01 seconds

--- Running with Seed: 2 ---
--- Seed 2 Results ---
Success Rate: 0.3600 (36.00%)
Average Detection Time (Successful Episodes): 2.89
Average Episode Reward